# OmniQ Component-Based Usage Examples

This notebook demonstrates the component-based approach to using the OmniQ library, where each component (Queue, Result Store, Worker) is instantiated and used separately.

## Overview

The component-based approach provides maximum flexibility by allowing you to:

- **Decouple Components**: Create and configure each component independently
- **Mix and Match Backends**: Use different storage backends for different components
- **Fine-Grained Control**: Configure each component with specific settings
- **Modular Architecture**: Replace or upgrade individual components without affecting others
- **Direct Component Access**: Work directly with component APIs for advanced use cases

This approach is ideal when you need maximum flexibility and control over your task processing system.

## Setup and Imports

First, let's import the individual components we'll be working with:

In [ ]:
import datetime as dt
from omniq.queue import FileTaskQueue, SQLiteTaskQueue
from omniq.results import SQLiteResultStorage, FileResultStorage
from omniq.events import PostgresEventStorage
from omniq.workers import ThreadPoolWorker, AsyncWorker

## Basic Component-Based Usage

Let's start with the fundamental component-based approach where we create each component individually:

In [ ]:
# Create components individually
queue = FileTaskQueue(
    project_name="my_project",
    base_dir="some/path",
    queues=["low", "medium", "high"]
)

result_store = SQLiteResultStorage(
    project_name="my_project",
    base_dir="some/path"
)

print("Created individual components:")
print(f"- Task Queue: {type(queue).__name__}")
print(f"- Result Store: {type(result_store).__name__}")

### Creating a Worker with Component References

Now we create a worker that references our queue and result store:

In [ ]:
# Create worker with reference to queue and result store
worker = ThreadPoolWorker(
    queue=queue,
    result_store=result_store,
    max_workers=20
)

print(f"Created worker: {type(worker).__name__} with {worker.max_workers} max workers")

### Defining and Running Tasks

Let's define a simple task and see how the components work together:

In [ ]:
# Define a task
def simple_task(name):
    print(f"Hello {name}")
    return name

print("Defined simple_task function")

In [ ]:
# Start the worker
worker.start()
print("Worker started")

# Enqueue a task
task_id = queue.enqueue(
    func=simple_task,
    func_args=dict(name="Tom"),
    queue_name="low"
)

print(f"Enqueued task with ID: {task_id}")

# Get the result
result = result_store.get(task_id)
print(f"Task result: {result}")

# Stop the worker
worker.stop()
print("Worker stopped")

## Context Manager Approach

For proper resource management, it's recommended to use context managers. This ensures automatic cleanup of resources:

In [ ]:
# Using context managers for automatic resource cleanup
with FileTaskQueue(
    project_name="my_project",
    base_dir="some/path",
    queues=["low", "medium", "high"]
) as queue, SQLiteResultStorage(
    project_name="my_project",
    base_dir="some/path"
) as result_store, ThreadPoolWorker(
    queue=queue, 
    result_store=result_store,
    max_workers=10
) as worker:
    
    print("All components initialized with context managers")
    
    task_id = queue.enqueue(simple_task, func_args=dict(name="Tom"))
    print(f"Enqueued task with ID: {task_id}")
    
    result = result_store.get(task_id)
    print(f"Context manager result: {result}")

print("Context managers automatically cleaned up resources")

## Mixed Backends Example

One of the key advantages of the component-based approach is the ability to mix different backends for different components. This allows you to optimize each component for its specific use case:

In [ ]:
# Define a task for mixed backends example
def processing_task(name):
    print(f"Processing {name} with mixed backends")
    return f"Processed: {name}"

print("Defined processing_task function")

In [ ]:
# Mix different backends:
# - File-based task queue for simplicity
# - File result storage for portability  
# - PostgreSQL event storage for advanced querying

mixed_queue = FileTaskQueue(
    project_name="mixed_example",
    base_dir="./queue_storage",
    queues=["processing"]
)

mixed_result_store = FileResultStorage(
    project_name="mixed_example", 
    base_dir="./result_storage"
)

mixed_event_store = PostgresEventStorage(
    project_name="mixed_example",
    host="localhost",
    port=5432,
    username="postgres",
    password="secret"
)

print("Created mixed backend components:")
print(f"- Queue: {type(mixed_queue).__name__} (File-based)")
print(f"- Results: {type(mixed_result_store).__name__} (File-based)")
print(f"- Events: {type(mixed_event_store).__name__} (PostgreSQL-based)")

In [ ]:
# Use async worker for better I/O performance
mixed_worker = AsyncWorker(
    queue=mixed_queue,
    result_store=mixed_result_store,
    event_store=mixed_event_store,
    max_workers=5
)

print(f"Created {type(mixed_worker).__name__} with event logging")

In [ ]:
# Run the mixed backends example
with mixed_queue, mixed_result_store, mixed_event_store, mixed_worker:
    # Enqueue multiple tasks
    task_ids = []
    for i in range(3):
        task_id = mixed_queue.enqueue(
            func=processing_task,
            func_args=dict(name=f"Task-{i+1}"),
            queue_name="processing"
        )
        task_ids.append(task_id)
        print(f"Enqueued task {i+1} with ID: {task_id}")

    # Get results
    for task_id in task_ids:
        result = mixed_result_store.get(task_id)
        print(f"Result for {task_id}: {result}")

print("Mixed backends example completed")

## Priority Queue Processing

The component-based approach makes it easy to implement priority-based task processing by using multiple named queues:

In [ ]:
# Define tasks with different priorities
def urgent_task(message):
    print(f"🚨 URGENT: {message}")
    return f"urgent_result: {message}"

def normal_task(message):
    print(f"📋 Normal: {message}")
    return f"normal_result: {message}"

def low_priority_task(message):
    print(f"⏳ Low priority: {message}")
    return f"low_result: {message}"

print("Defined priority-based task functions")

In [ ]:
# Create SQLite queue with priority queues
priority_queue = SQLiteTaskQueue(
    project_name="priority_example",
    base_dir="./priority_storage",
    queues=["high", "medium", "low"]  # Worker processes in this order
)

priority_result_store = SQLiteResultStorage(
    project_name="priority_example",
    base_dir="./priority_storage"
)

priority_worker = ThreadPoolWorker(
    queue=priority_queue,
    result_store=priority_result_store,
    max_workers=3
)

print("Created priority queue system with SQLite backend")
print("Queue priority order: high → medium → low")

In [ ]:
# Demonstrate priority processing
with priority_queue, priority_result_store, priority_worker:
    task_ids = []
    
    # Enqueue tasks in mixed order - worker will process by priority
    
    # Enqueue low priority first
    low_id = priority_queue.enqueue(
        func=low_priority_task,
        func_args=dict(message="This should run last"),
        queue_name="low"
    )
    task_ids.append(("low", low_id))
    
    # Then normal priority
    normal_id = priority_queue.enqueue(
        func=normal_task,
        func_args=dict(message="This should run second"),
        queue_name="medium"
    )
    task_ids.append(("medium", normal_id))
    
    # Finally high priority (but will be processed first)
    high_id = priority_queue.enqueue(
        func=urgent_task,
        func_args=dict(message="This should run first"),
        queue_name="high"
    )
    task_ids.append(("high", high_id))

    print("Tasks enqueued. Worker will process high → medium → low priority.")
    
    # Get results
    for priority, task_id in task_ids:
        result = priority_result_store.get(task_id)
        print(f"[{priority}] Result: {result}")

print("Priority queue example completed")

## Key Benefits of Component-Based Approach

### 1. **Flexibility**
- Choose the best backend for each component based on your specific requirements
- Mix file-based queues with database result storage
- Use different databases for different components

### 2. **Scalability** 
- Scale components independently
- Run multiple workers with shared storage
- Distribute components across different servers

### 3. **Testing**
- Mock individual components for unit testing
- Test components in isolation
- Use memory backends for fast testing

### 4. **Migration**
- Migrate components one at a time
- No system-wide changes required
- Gradual adoption of new backends

### 5. **Performance Optimization**
- Configure each component for its specific workload
- Use fast storage for frequently accessed data
- Use durable storage for critical data

## When to Use Component-Based Approach

Consider the component-based approach when you need:

- **Different storage backends** for different components
- **Fine-grained control** over component configuration
- **Integration** with existing systems
- **Maximum flexibility** and customization
- **Custom orchestration logic** around OmniQ components
- **Independent scaling** of different components

For simpler use cases, consider:
- **Basic usage** examples for straightforward task processing
- **Backend-based configuration** for unified storage across all components
- **Configuration-based setup** for environment-driven configuration

## Next Steps

Now that you understand the component-based approach, explore these advanced topics:

- **Backend-based usage**: Use unified backends for simplified configuration
- **Configuration-based usage**: Load settings from YAML files or environment variables
- **Cloud storage integration**: Use S3, Azure, or GCP for distributed processing
- **Custom worker implementations**: Create specialized workers for specific workloads
- **Event-driven architectures**: Build reactive systems using OmniQ's event system
- **Monitoring and observability**: Implement comprehensive task monitoring

The component-based approach provides the foundation for building sophisticated, scalable task processing systems!